In [1]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam


In [2]:
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.15.0


In [3]:
# Function to load images and labels from a directory
def load_images_from_directory(directory):
    images = []
    labels = []
    label_map = {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'sad': 4, 'surprise': 5, 'neutral': 6}
    for label in label_map:
        label_directory = os.path.join(directory, label)
        for filename in os.listdir(label_directory):
            img_path = os.path.join(label_directory, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img = cv2.resize(img, (48, 48))
                images.append(img)
                labels.append(label_map[label])
    return np.array(images), np.array(labels)

In [4]:
# Load images and labels from the train and test directories
train_directory = "FacialExpression/train"
test_directory = "FacialExpression/test"
X_train, y_train = load_images_from_directory(train_directory)
X_test, y_test = load_images_from_directory(test_directory)

# Split the dataset into train, validation, and test sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

# Preprocess the   
X_train = X_train.reshape((-1, 48, 48, 1)) / 255.0
X_val = X_val.reshape((-1, 48, 48, 1)) / 255.0
X_test = X_test.reshape((-1, 48, 48, 1)) / 255.0
y_train = to_categorical(y_train, num_classes=7)
y_val = to_categorical(y_val, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)



In [5]:
# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(7, activation='softmax') 
])


In [6]:
# Compile the model

model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))



Epoch 1/10


808/808 [==============================] - 27s 30ms/step - loss: 1.6248 - accuracy: 0.3572 - val_loss: 1.4415 - val_accuracy: 0.4364
Epoch 2/10
808/808 [==============================] - 27s 34ms/step - loss: 1.3542 - accuracy: 0.4842 - val_loss: 1.3165 - val_accuracy: 0.4970
Epoch 3/10
808/808 [==============================] - 28s 35ms/step - loss: 1.2369 - accuracy: 0.5325 - val_loss: 1.2732 - val_accuracy: 0.5131
Epoch 4/10
808/808 [==============================] - 24s 29ms/step - loss: 1.1583 - accuracy: 0.5641 - val_loss: 1.2489 - val_accuracy: 0.5392
Epoch 5/10
808/808 [==============================] - 23s 29ms/step - loss: 1.0873 - accuracy: 0.5932 - val_loss: 1.1961 - val_accuracy: 0.5347
Epoch 6/10
808/808 [==============================] - 23s 29ms/step - loss: 1.0167 - accuracy: 0.6220 - val_loss: 1.2180 - val_accuracy: 0.5455
Epoch 7/10
808/808 [==============================] - 23s 29ms/step - loss: 0.9460 - accuracy: 0.6451 - val_loss: 1.2022 - val_accurac

In [8]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)


225/225 [==============================] - 3s 12ms/step - loss: 1.2682 - accuracy: 0.5731
Test accuracy: 0.5731401443481445


In [9]:
from tensorflow.keras.optimizers import Adam

In [10]:
model.save("facial_expression_model.h5")

C:\Users\syedj\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [11]:
model = tf.keras.models.load_model('facial_expression_model.h5')

In [12]:
def resize(img, size):
    width = int(img.shape[1] * size)
    height = int(img.shape[0] * size)
    dimension = (width, height)
    return cv2.resize(img, dimension, interpolation=cv2.INTER_AREA)

In [13]:
# Function to find face encodings for recognition
def findEncoding(images):
    imgEncodings = []
    for img in images:
        img = cv2.resize(img, (0, 0), fx=0.5, fy=0.5)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encodeimg = face_recognition.face_encodings(img)[0]  # Corrected the function name
        imgEncodings.append(encodeimg)
    return imgEncodings


In [14]:
import cv2
import numpy as np
import face_recognition as face_rec
import os
from datetime import datetime

In [15]:
import cv2
import numpy as np
import os
import csv
import datetime
import tensorflow as tf
import face_recognition

# Load face cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load student images and names for recognition
path = 'student_images'
studentImg = []
studentName = []
myList = os.listdir(path)
for cl in myList:
    curimg = cv2.imread(f'{path}/{cl}')
    studentImg.append(curimg)
    studentName.append(os.path.splitext(cl)[0])

# Function to find face encodings for recognition
def findEncoding(images):
    imgEncodings = []
    for img in images:
        img = cv2.resize(img, (0, 0), fx=0.5, fy=0.5)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encodeimg = face_recognition.face_encodings(img)[0]
        imgEncodings.append(encodeimg)
    return imgEncodings

# Get face encodings for student images
EncodeList = findEncoding(studentImg)

# Load emotion detection model
model = tf.keras.models.load_model('facial_expression_model.h5')  # Load your emotion detection model here

# Define emotion labels
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Open CSV file for writing and write headers
with open('students_data.csv', mode='a', newline='') as file:  # Append mode
    writer = csv.writer(file)
    # Check if the file is empty
    if file.tell() == 0:
        writer.writerow(['Name', 'Expression', 'Date', 'Time'])  # Write headers

    # Start video capture
    vid = cv2.VideoCapture(0)
    while True:
        success, frame = vid.read()
        if not success:
            break

        # Get current date and time
        current_date = datetime.datetime.now().date()
        current_time = datetime.datetime.now().strftime('%H:%M:%S')

        # Convert frame to grayscale for face detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the frame
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in faces:
            # Extract face ROI
            face_roi = gray[y:y + h, x:x + w]

            # Resize to match model input size for emotion detection
            face_roi_resized = cv2.resize(face_roi, (48, 48))
            face_roi_resized = np.expand_dims(face_roi_resized, axis=0)
            face_roi_resized = np.expand_dims(face_roi_resized, axis=-1)

            # Predict emotion
            predicted_emotion = model.predict(face_roi_resized)
            emotion_label = emotion_labels[np.argmax(predicted_emotion)]

            # Recognize student and add to CSV
            face_gray = gray[y:y+h, x:x+w]
            face_rgb = frame[y:y+h, x:x+w]
            Smaller_frame = cv2.resize(face_rgb, (0, 0), None, 0.25, 0.25)

            facesInFrame = face_rec.face_locations(Smaller_frame)
            encodeFacesInFrame = face_rec.face_encodings(Smaller_frame, facesInFrame)

            for encodeFace, faceLoc in zip(encodeFacesInFrame, facesInFrame):
                matches = face_rec.compare_faces(EncodeList, encodeFace)
                faceDis = face_rec.face_distance(EncodeList, encodeFace)
                print(faceDis)
                matchIndex = np.argmin(faceDis)

                if matches[matchIndex]:
                    name = studentName[matchIndex].upper()
                    writer.writerow([name, emotion_label, current_date, current_time])

                    # Draw rectangle around the face and label the emotion and name
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
                    cv2.putText(frame, f'{name}: {emotion_label}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Display the resulting frame
        cv2.imshow('Facial Expression and Person Identification', frame)

        # Exit if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        # Flush changes to the CSV file to ensure auto-saving
        file.flush()

    # Release the video capture
    vid.release()


1/1 [==============================] - 0s 51ms/step


1/1 [==============================] - 0s 46ms/step


1/1 [==============================] - 0s 47ms/step


1/1 [==============================] - 0s 48ms/step


1/1 [==============================] - 0s 31ms/step
[0.66671421 0.76457064 0.68836094 0.6938033 ]
1/1 [==============================] - 0s 27ms/step
[0.67787273 0.71316028 0.30784382 0.75710805]


1/1 [==============================] - 0s 30ms/step
[0.68057014 0.74243381 0.28861549 0.75075853]
1/1 [==============================] - 0s 26ms/step
[0.69365895 0.76321551 0.27985043 0.73581194]
1/1 [==============================] - 0s 27ms/step
[0.674591   0.74370845 0.33815267 0.7460134 ]
1/1 [==============================] - 0s 26ms/step
[0.64970266 0.75200003 0.26823504 0.7406622 ]
1/1 [==============================] - 0s 26ms/step
[0.70390944 0.75733869 0.3315847  0.76352858]
1/1 [==============================] - 0s 26ms/step
[0.71424131 0.76921796 0.34529056 0.76951472]
1/1 [==============================] - 0s 27ms/step
[0.68576207 0.75964567 0.29977252 0.76023478]
1/1 [==============================] - 0s 25ms/step
[0.73770918 0.79297676 0.33009083 0.78330012]
1/1 [==============================] - 0s 25ms/step
[0.75883579 0.78395407 0.32617469 0.78240169]
1/1 [==============================] - 0s 27ms/step
[0.72382278 0.80771812 0.35160412 0.78627472]
1/1 [===============

1/1 [==============================] - 0s 31ms/step
[0.72451606 0.43877488 0.71518086 0.6947392 ]
1/1 [==============================] - 0s 31ms/step
[0.69976703 0.31284896 0.67600089 0.65946587]
1/1 [==============================] - 0s 26ms/step
[0.68959991 0.32943714 0.68412403 0.64384093]
1/1 [==============================] - 0s 26ms/step
[0.69570299 0.32916271 0.69107095 0.62697081]
1/1 [==============================] - 0s 25ms/step
[0.69166643 0.32995363 0.69671786 0.6423474 ]
1/1 [==============================] - 0s 27ms/step
[0.68821836 0.33412734 0.68336706 0.66595902]
1/1 [==============================] - 0s 25ms/step
[0.69619574 0.3955702  0.66366364 0.66187244]
1/1 [==============================] - 0s 27ms/step
[0.67116728 0.38174415 0.63936816 0.6588038 ]
1/1 [==============================] - 0s 31ms/step
[0.69252492 0.39317572 0.66695108 0.62759781]
1/1 [==============================] - 0s 25ms/step
[0.71789686 0.33875943 0.72515093 0.66229682]
1/1 [===============

[0.65518081 0.39739752 0.71545091 0.59323357]
1/1 [==============================] - 0s 48ms/step
[0.65430583 0.40571485 0.71482764 0.62550594]
1/1 [==============================] - 0s 46ms/step
[0.6918421  0.42978602 0.70291567 0.64583613]
1/1 [==============================] - 0s 48ms/step
[0.70580845 0.43945021 0.76270796 0.65805489]
1/1 [==============================] - 0s 53ms/step
[0.70170517 0.39762347 0.69780945 0.61963345]
1/1 [==============================] - 0s 52ms/step
[0.66254832 0.35250216 0.71887712 0.59970034]
1/1 [==============================] - 0s 52ms/step
[0.67256917 0.40822463 0.69675756 0.60480432]
1/1 [==============================] - 0s 46ms/step
[0.60826114 0.39115917 0.67868172 0.55602119]
1/1 [==============================] - 0s 52ms/step
[0.57354317 0.42135984 0.68479069 0.59561696]
1/1 [==============================] - 0s 49ms/step
[0.67978029 0.45154979 0.67850914 0.61145678]
1/1 [==============================] - 0s 50ms/step
[0.61890675 0.315760

1/1 [==============================] - 0s 47ms/step


KeyboardInterrupt: 